# IA para Manuteção Preditiva usando Redes Neurais

### Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline  

### Importar Dataset

Segue abaixo link para acessar a base de dados:

https://www.kaggle.com/behrad3d/nasa-cmaps

In [ ]:
dataset_train=pd.read_csv('../input/PM_train.txt',sep=' ',header=None).drop([26,27],axis=1)
col_names = ['id','cycle','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
dataset_train.columns=col_names
print('Ver dataset de treino: ',dataset_train.shape)
dataset_train.head()

In [ ]:
dataset_test=pd.read_csv('../input/PM_test.txt',sep=' ',header=None).drop([26,27],axis=1)
dataset_test.columns=col_names
print('Ver dataset de teste: ',dataset_train.shape)
dataset_train.head()

In [ ]:
pm_truth=pd.read_csv('../input/PM_truth.txt',sep=' ',header=None).drop([1],axis=1)
pm_truth.columns=['more']
pm_truth['id']=pm_truth.index+1
pm_truth.head()

In [ ]:
# gerar coluna máxima para dados de teste
rul = pd.DataFrame(dataset_test.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
rul.head()

In [ ]:
# rodando a falha 
pm_truth['rtf']=pm_truth['more']+rul['max']
pm_truth.head()

In [ ]:
#Trazer uma coluna ciclo de trás para frente para identificar o ciclo da falha e poder prever em quantidade de dias a falha
pm_truth.drop('more', axis=1, inplace=True)
dataset_test=dataset_test.merge(pm_truth,on=['id'],how='left')
dataset_test['ttf']=dataset_test['rtf'] - dataset_test['cycle']
dataset_test.drop('rtf', axis=1, inplace=True)
dataset_test.head()

In [ ]:
dataset_train['ttf'] = dataset_train.groupby(['id'])['cycle'].transform(max)-dataset_train['cycle']
dataset_train.head()

In [ ]:
#Previsão de período de quebra e identifica-lo (periodo determido na váriavel 'period') em 0 e 1
df_train=dataset_train.copy()
df_test=dataset_test.copy()
period=30
df_train['label_bc'] = df_train['ttf'].apply(lambda x: 1 if x <= period else 0)
df_test['label_bc'] = df_test['ttf'].apply(lambda x: 1 if x <= period else 0)
df_train.head()

In [ ]:
features_col_name=['setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11',
                   's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
target_col_name='label_bc'

## Feature Scaling

In [ ]:
sc=MinMaxScaler()
df_train[features_col_name]=sc.fit_transform(df_train[features_col_name])
df_test[features_col_name]=sc.transform(df_test[features_col_name])

## Função para remodelar o conjunto de dados conforme exigido pela Rede Neural LSTM

In [ ]:
def gen_sequence(id_df, seq_length, seq_cols):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        lstm_array.append(data_array[start:stop, :])
    return np.array(lstm_array)

# função para gerar labels
def gen_label(id_df, seq_length, seq_cols,label):
    df_zeros=pd.DataFrame(np.zeros((seq_length-1,id_df.shape[1])),columns=id_df.columns)
    id_df=df_zeros.append(id_df,ignore_index=True)
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    y_label=[]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        y_label.append(id_df[label][stop])
    return np.array(y_label)

In [ ]:
# timestamp or window size
seq_length=50
seq_cols=features_col_name

In [ ]:
# gerando X_train
X_train=np.concatenate(list(list(gen_sequence(df_train[df_train['id']==id], seq_length, seq_cols)) for id in df_train['id'].unique()))
print(X_train.shape)
# gerando y_train
y_train=np.concatenate(list(list(gen_label(df_train[df_train['id']==id], 50, seq_cols,'label_bc')) for id in df_train['id'].unique()))
print(y_train.shape)

In [ ]:
# gerando X_test
X_test=np.concatenate(list(list(gen_sequence(df_test[df_test['id']==id], seq_length, seq_cols)) for id in df_test['id'].unique()))
print(X_test.shape)
# gerando y_test
y_test=np.concatenate(list(list(gen_label(df_test[df_test['id']==id], 50, seq_cols,'label_bc')) for id in df_test['id'].unique()))
print(y_test.shape)

## Desenvolvendo Rede Neural LSTM Network

Vamos utilizar a rede Neural LSTM, ao usa-las no domínio da série temporal, um parâmetro importante é o comprimento da sequência, a janela para examinar o sinal de falha.
Isso pode ser visto como a escolha de um window_size (ou seja, 5 ciclos) para calcular os recursos de rolagem no modelo de manutenção preditiva, eles incluíram a média de rolagem e o desvio padrão de rolagem ao longo dos 5 ciclos para cada um dos 21 valores do sensor e no deep learning, permitimos que o LSTM extraia recursos abstratos da sequência de valores do sensor dentro da janela, sendo que a expectativa é que os padrões dentro desses valores sejam codificados automaticamente pelo LSTM.

Outra vantagem é que sua capacidade de lembrar de sequências de longo prazo, o que é difícil de conseguir pelos recursos tradicionais. Calcular médias contínuas em um tamanho de janela de 50 ciclos pode levar à perda de informações devido à suavização em um período tão longo, já o LSTM pode usar tamanhos de janela maiores e usar todas as informações da janela como entrada.

Neste cenário vamos construir uma rede de apenas 2 camadas, com dropout. A primeira camada LSTM com 100 unidades (uma para cada sequência de entrada) seguida por outra camada LSTM com 50 unidades.
Também aplicaremos a eliminação de cada camada LSTM para controlar o sobreajuste e a camada de saída densa final utiliza-se uma ativação sigmóide.

Para saber mais sobre a rede neural LSTM acesse o link:
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

Esta rede Neural é baseada na do link abaixo:
https://github.com/Azure-Samples/MachineLearningSamples-DeepLearningforPredictiveMaintenance/blob/master/Code/2_model_building_and_evaluation.ipynb


In [ ]:
nb_features =X_train.shape[2]
timestamp=seq_length

model = Sequential()

model.add(LSTM(
         input_shape=(timestamp, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
# definindo neurônios (metodo fit)
model.fit(X_train, y_train, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

## Teste de Performance da Rede Neural

Pdemos ver que a acuracia do modelo de treino foi muito boa chegando a 96,6%, já a acurácia do modelo teste chegou á 99% com um AUC (área sob a curva) de 92%.

In [ ]:
import scikitplot as skplt
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import classification_report

# treinando metricas
scores = model.evaluate(X_train, y_train, verbose=1, batch_size=200)
print('Accurracy do modelo nos dados de treino: {}'.format(scores[1]))

y_pred=model.predict_classes(X_test)
print('\n[Classification Report]')
print("Accuracy do modelo nos dados de teste: {:.2f}".format(accuracy_score(y_test,y_pred)))
# imprimir a área sob da curva AUC
print("AUC: {:.2f}\n".format(roc_auc_score(y_test,y_pred)))

print(classification_report(y_test,y_pred))


skplt.metrics.plot_confusion_matrix(y_test,y_pred, normalize=True)

### Probabilidade de falha

Iremos prever a probabilidade de quebras das machines(maquina/turninas) em 30 dias, onde na váriavel machine_id, identifico a ID do equipamento.

In [ ]:
def prob_failure(machine_id):
    machine_df=df_test[df_test.id==machine_id]
    machine_test=gen_sequence(machine_df,seq_length,seq_cols)
    m_pred=model.predict(machine_test)
    failure_prob=list(m_pred[-1]*100)[0]
    return failure_prob

machine_id=20
print('Probabilidade de falha em 30 dias: ',prob_failure(machine_id))